In [4]:
import os
import cv2
import sys
from PyQt5.QtWidgets import (
    QApplication,
    QLabel,
    QMainWindow,
    QPushButton,
    QWidget,
    QVBoxLayout,
    QHBoxLayout,
    QCheckBox,
)
from PyQt5.QtGui import QPixmap, QTransform
from PyQt5.QtCore import Qt
import csv
import datetime

### Params


In [3]:
primary_label = "natural"  # Một khi chọn nhãn này thì sẽ clear các nhãn còn lại trong lúc đánh nhãn bằng tool
skip_frame_by_s = 0.2  # skip 0.2 second

label_list_dict = [
    "natural",
    "sleepy_eye",  # Mắt nhắm hoặc lờ đờ (dấu hiệu buồn ngủ)
    "yawn",  # ngáp
    "rub_eye",  # dụi mắt
    "look_away",
]

videos_source_folder = "../old_assets"
current_label = "natural"

In [3]:
# Convert datetime to string
def convert_datetime_to_string(datetime):
    return datetime.strftime("%Y%m%d_%H%M%S")

### Trích xuất video thành các ảnh( mỗi ảnh cách nhau 0.5s) và chuyển vào 1 folder


In [4]:
def extract_images_from_video(
    video_path: str,
    output_folder: str,
    time_step: float = 0.25,
    is_rotate: bool = False,
):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lấy tên file gốc không bao gồm phần mở rộng
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS): {fps}")

    frame_interval = int(fps * time_step)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if is_rotate:
            frame = cv2.rotate(
                frame, cv2.ROTATE_90_COUNTERCLOCKWISE
            )  # Xoay 90 độ ngược chiều kim đồng hồ

        if frame_count % frame_interval == 0:
            saved_count += 1
            # Định dạng tên file: <tên file gốc>_<số thứ tự dạng 3 chữ số>.jpg

            output_file = os.path.join(
                output_folder, f"{base_name}_{saved_count:03d}.jpg"
            )
            cv2.imwrite(output_file, frame)
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [ ]:

frames_extraction_folders = []

# Lấy danh sách các video trong thư mục
video_files = os.listdir(videos_source_folder)

for video_path in video_files:
    if not video_path.endswith(".mp4"):
        continue

    video_name = video_path.split(".")[0]
    current_date_time = convert_datetime_to_string(datetime.datetime.now())
    original_video_path = f"{videos_source_folder}/{video_path}"

    frames_extraction_path = f"frames_extraction/{current_date_time}"

    frames_extraction_folders.append(frames_extraction_path)

    print("-> Processing video: ", video_path)

    extract_images_from_video(
        original_video_path, frames_extraction_path, skip_frame_by_s, is_rotate=True
    )

-> Processing video:  6382603217629.mp4
Frames per second using video.get(cv2.CAP_PROP_FPS): 30.0
-> Processing video:  6382664702725.mp4
Frames per second using video.get(cv2.CAP_PROP_FPS): 30.0
-> Processing video:  6382674313972.mp4
Frames per second using video.get(cv2.CAP_PROP_FPS): 30.0


### Tool


In [5]:
def copy_images_to_folder(
    input_file: str, output_folder: str, angle: int, folder_name: str, image_stt: int
) -> str:
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pixmap = QPixmap(input_file)
    transform = QTransform().rotate(angle)
    rotated_pixmap = pixmap.transformed(transform)

    # Format STT thành 3 chữ số (001, 002, ...)
    stt_str = f"{image_stt:03d}"

    _, ext = os.path.splitext(os.path.basename(input_file))
    new_filename = f"{folder_name}_{stt_str}{ext}"  # Ví dụ: "20250307_222829_001.jpg"
    output_file = os.path.join(output_folder, new_filename)

    rotated_pixmap.save(output_file)

    return new_filename

In [6]:
def multi_labels_tool(
    input_folder: str,
    output_folder: str,
    label_list: list,
    primary_label: str,
    csv_file_name: str,
):
    current_angle = 0

    app = QApplication(sys.argv)
    main_window = QMainWindow()
    main_window.setGeometry(500, 100, 1000, 900)
    main_window.setWindowTitle("Labeling Tool")
    main_window.setStyleSheet("background-color: black;")

    images = os.listdir(input_folder)
    image_count = len(images)
    current_image_index = 0

    # Tạo folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    central_widget = QWidget()
    main_window.setCentralWidget(central_widget)

    layout = QVBoxLayout()
    central_widget.setLayout(layout)

    image_checkbox_layout = QHBoxLayout()
    layout.addLayout(image_checkbox_layout)

    name_pic = QLabel()
    name_pic.setStyleSheet("color: white;")
    layout.addWidget(name_pic)

    pixmap = QPixmap(f"{input_folder}/{images[current_image_index]}")

    # Lấy kích thước cửa sổ chính
    window_width = main_window.width()
    window_height = main_window.height()

    # Giới hạn kích thước tối đa của ảnh (80% kích thước cửa sổ)
    max_width = int(window_width * 0.8)
    max_height = int(window_height * 0.8)

    # Scale ảnh mà giữ nguyên tỷ lệ
    scaled_pixmap = pixmap.scaled(
        max_width, max_height, Qt.KeepAspectRatio, Qt.SmoothTransformation
    )

    label = QLabel()
    label.setPixmap(scaled_pixmap)
    label.setMaximumSize(max_width, max_height)
    label.setScaledContents(True)
    image_checkbox_layout.addWidget(label)

    checkbox_layout = QVBoxLayout()
    checkbox_layout.setContentsMargins(50, 0, 0, 0)
    image_checkbox_layout.addLayout(checkbox_layout)

    checkboxes = []
    for label_name in label_list:
        checkbox = QCheckBox(label_name)
        checkbox.setStyleSheet("color: white;")
        checkbox_layout.addWidget(checkbox)
        checkboxes.append(checkbox)

        def on_checkbox_state_changed(state, current_checkbox=checkbox):
            if current_checkbox.text() == primary_label and state == Qt.Checked:
                for other_checkbox in checkboxes:
                    if other_checkbox != current_checkbox:
                        other_checkbox.setChecked(False)
            if current_checkbox.text() != primary_label and state == Qt.Checked:
                for other_checkbox in checkboxes:
                    if other_checkbox.text() == primary_label:
                        other_checkbox.setChecked(False)

        checkbox.stateChanged.connect(on_checkbox_state_changed)

    label_status = QLabel()
    label_status.setStyleSheet("color: white;")
    layout.addWidget(label_status)

    def rotate_image(angle):
        nonlocal pixmap
        transform = QTransform().rotate(angle)
        rotated_pixmap = pixmap.transformed(transform)
        label.setPixmap(rotated_pixmap)

    def update_ui():
        nonlocal current_image_index, current_angle
        pixmap.load(f"{input_folder}/{images[current_image_index]}")
        label.setPixmap(pixmap)
        name_pic.setText(
            f"{current_image_index + 1}/{image_count} - {images[current_image_index]}"
        )
        rotate_image(current_angle)

    update_ui()

    button_layout = QHBoxLayout()
    layout.addLayout(button_layout)

    btn_back = QPushButton("Back")
    btn_back.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_back)

    def btn_back_clicked():
        nonlocal current_image_index
        current_image_index -= 1
        if current_image_index < 0:
            current_image_index = len(images) - 1
        for checkbox in checkboxes:
            checkbox.setChecked(False)
        update_ui()

    btn_back.clicked.connect(btn_back_clicked)

    btn_continue = QPushButton("Continue")
    btn_continue.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_continue)

    def btn_continue_clicked():
        nonlocal current_image_index
        current_image_index += 1
        if current_image_index >= len(images):
            current_image_index = 0
        for checkbox in checkboxes:
            checkbox.setChecked(False)
        update_ui()

    btn_continue.clicked.connect(btn_continue_clicked)

    btn_save = QPushButton("Save and continue (click S or s)")
    btn_save.setStyleSheet("background-color: white;")
    layout.addWidget(btn_save)

    def btn_save_clicked():
        nonlocal current_image_index
        image_name = images[current_image_index]

        selected_labels = [
            checkbox.text() for checkbox in checkboxes if checkbox.isChecked()
        ]

        if not selected_labels:
            label_status.setText("No label selected")
            return

        # Lấy tên thư mục từ `input_folder`
        folder_name = os.path.basename(input_folder)

        # Lưu ảnh đã xoay với tên mới và lấy tên file mới
        new_image_name = copy_images_to_folder(
            f"{input_folder}/{image_name}",
            f"{output_folder}",
            current_angle,
            folder_name,
            current_image_index + 1,  # STT bắt đầu từ 1
        )

        # Xây dựng hàng dữ liệu: [Tên file mới, Danh sách lỗi, Natural, Yawn, Sleepy_Eyes, Rub_Eye, Look_Away]
        row_data = [f"images/{new_image_name}", ", ".join(selected_labels)] + [
            1 if checkbox.isChecked() else 0 for checkbox in checkboxes
        ]

        # Kiểm tra nếu file CSV chưa tồn tại, thì ghi header
        file_exists = os.path.exists(csv_file_name)

        with open(csv_file_name, mode="a", newline="") as file:
            writer = csv.writer(file)

            # Nếu file chưa có, ghi header trước
            if not file_exists:
                writer.writerow(["Filename", "Errors"] + label_list)

            # Ghi dữ liệu của ảnh hiện tại
            writer.writerow(row_data)

        label_status.setText(
            f"Copied {new_image_name} to folders: {', '.join(selected_labels)}"
        )

        for checkbox in checkboxes:
            checkbox.setChecked(False)

        btn_continue_clicked()

    btn_save.clicked.connect(btn_save_clicked)

    def key_press_event(event):
        nonlocal current_image_index
        if event.key() in [Qt.Key_S, Qt.Key_S]:
            btn_save_clicked()

    main_window.keyPressEvent = key_press_event

    btn_rotate_left = QPushButton("Rotate Left")
    btn_rotate_left.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_rotate_left)

    def btn_rotate_left_clicked():
        nonlocal current_angle
        current_angle -= 90
        if current_angle < 0:
            current_angle = 270
        rotate_image(current_angle)

    btn_rotate_left.clicked.connect(btn_rotate_left_clicked)

    btn_rotate_right = QPushButton("Rotate Right")
    btn_rotate_right.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_rotate_right)

    def btn_rotate_right_clicked():
        nonlocal current_angle
        current_angle += 90
        if current_angle >= 360:
            current_angle = 0
        rotate_image(current_angle)

    btn_rotate_right.clicked.connect(btn_rotate_right_clicked)

    main_window.show()
    try:
        sys.exit(app.exec_())
    except:
        print("Exiting")

#### Đôi khi kernel bị die nên cần gán thủ công input_folder

Ví dụ: window(`'frames_extraction/20250307_210625`, data_output_folder, primary_label)


In [ ]:
# Danh sách folder trong `frames_extraction`
os_folders = os.listdir("frames_extraction")

In [8]:
os_folders

['20250307_222829',
 '20250307_222836',
 '20250307_222838',
 '20250307_222846',
 '20250307_222852',
 '20250307_222859',
 '20250321_221026',
 '20250321_221029',
 '20250321_221034']

In [11]:
# Dựa vào đường dẫn của các folder trích xuất ảnh
data_output_folder = f"multi_label_data/images"
if not os.path.exists(data_output_folder):
    os.makedirs(data_output_folder)

# Ví dụ: 'frames_extraction/20250307_222829'
multi_labels_tool(
    "frames_extraction/20250307_222829",
    data_output_folder,
    label_list_dict,
    primary_label,
    "multi_label_data/training_data.csv",
)

Exiting
